In [ ]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List
import json
import time

rootpath = ".."

import transfer_logic.transfer_LD_weightinit as transfer_LD_weightinit
import transfer_logic.transfer_LD_ft as transfer_LD_ft
import transfer_logic.transfer_autoLSTM as transfer_autoLSTM

degree_sign = u'\N{DEGREE SIGN}'

In [ ]:
result_filename = f"{rootpath}/results/result_data/transfer_results.json"

with open(result_filename, "r") as f:
    results_data = json.load(f)

ft_percents = results_data["options"]["ft_percents"]
shuffle_seeds = results_data["options"]["seeds"]
transfer_methods = ["weight_initialization_LSTMDense", "weight_initialization_Dense", "weight_initialization_AutoLSTM"]

In [ ]:
def save_results(from_key, to_key, transfer_method, ft_percent, shuffle_seed, rmse=None, mae=None, mae_sd=None, training_time=None, datapoints=None):
    result_filename = f"{rootpath}/results/result_data/transfer_results.json"
    with open(result_filename, "r") as f:
        results_data = json.load(f)

    ft_percents = results_data["options"]["ft_percents"]
    shuffle_seeds = results_data["options"]["seeds"]

    if not to_key in results_data[transfer_method][from_key]:
        results_data[transfer_method][from_key][to_key] = {f"ft{int(x*100)}" : {
                    "rmse": [None]*len(shuffle_seeds),
                    "mae": [None]*len(shuffle_seeds),
                    "mae_sd": [None]*len(shuffle_seeds),
                    "training_time": [None]*len(shuffle_seeds),
                    "datapoints": 0
                } for x in ft_percents}
    
    saving_index = shuffle_seeds.index(shuffle_seed)
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["rmse"][saving_index] = rmse
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["mae"][saving_index] = mae
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["mae_sd"][saving_index] = mae_sd
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["training_time"][saving_index] = training_time
    results_data[transfer_method][from_key][to_key][f"ft{int(ft_percent*100)}"]["datapoints"] = datapoints
    
    with open(result_filename, "w") as f:
        json.dump(results_data, f, indent=4)

In [ ]:
# save_results("ESB1_summer", "Kissam1_summer", "weight_initialization_AutoLSTM", 0.2, 17)

In [ ]:
def plot_multiseed_error_pir(ft_percents: List[float], from_building: str, from_tower: int, from_season: str, to_building: str, to_tower: int, to_season: str = None, skip_existing_plot: bool = True):
    """
    Performs the following steps
    1. Performs transfer learning by finetuning entire model
    2. Performs transfer learning by finetuning only the output layer
    3. Plots mean absolute errors for test datasets picked with 5 different random seeds for both transfer learning methods
    4. Compares errors from transfer with error in the base model of the tower being transferred to
    5. Plots performance improvement ratio
    """
    to_features = [f'{to_building}_Tower_{to_tower} enteringWaterTemp', 
                    f'{to_building}_Tower_{to_tower} outdoorAirDryBulb', 
                    f'{to_building}_Tower_{to_tower} outdoorAirWetBulb', 
                    f'{to_building}_Tower_{to_tower} vfdPercent', 
                    f'{to_building}_Tower_{to_tower} fanA_vfdPower', 
                    f'{to_building}_Tower_{to_tower} fanB_vfdPower', 
                    f'{to_building}_Tower_{to_tower} dayOfWeek', 
                    f'{to_building}_Tower_{to_tower} hourOfDay', 
                    f'{to_building}_Tower_{to_tower} efficiency']
    to_target = f'{to_building}_Tower_{to_tower} leavingWaterTemp'
    
    for ft_percent in ft_percents:
            for s, seed in enumerate(shuffle_seeds):
                # perform each transfer learning method

                # finetuning all layers
                rmse1, _, mae1, tt, dp = transfer_LD_weightinit.transfer_LD_weightinit(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[0], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse1, mae=mae1, training_time=tt, datapoints=dp)

                # finetuning only output layer
                rmse2, _, mae2, tt, dp = transfer_LD_ft.transfer_LD_dense(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[1], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse2, mae=mae2, training_time=tt, datapoints=dp)

                # using lstm autoencoder model and finetuning only lstm layer
                rmse3, _, mae3, tt, dp = transfer_autoLSTM.transfer_autoLSTM(from_building_name=from_building, from_tower_number=from_tower, from_season=from_season, to_building_name=to_building, to_tower_number=to_tower, to_features=to_features, to_target=to_target, to_season=to_season, finetuning_percentage=ft_percent, display_results=False, use_delta=True, shuffle_seed=seed)
                save_results(from_key=f"{from_building}{from_tower}_{from_season}", to_key=f"{to_building}{to_tower}_{to_season}", transfer_method=transfer_methods[2], ft_percent=ft_percent, shuffle_seed=seed, rmse=rmse3, mae=mae3, training_time=tt, datapoints=dp)

In [ ]:
# plot_multiseed_error_pir(ft_percents=ft_percents, from_building="ESB", from_tower=1, from_season="summer", to_building="Kissam", to_tower=1, to_season="summer")

# Intra-season (Summer)

In [ ]:
# create all combinations of intraseason transfers for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
intraseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1==s2]
intraseason_combinations = list(set(intraseason_combinations)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
import random
random.seed(17)
intraseason_combinations = random.sample(intraseason_combinations, 5)
print(intraseason_combinations)

In [ ]:
for c in [intraseason_combinations[0]]:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

### Testing on MRB:

# create all combinations of interseason transfers between summer and fall for transferring from Kissam and ESB to MRB
buildings1 = ["ESB", "Kissam"]
buildings2 = ["MRB"]
towers = [1, 2]
seasons = ["summer", "fall"]
intraseason_combinations_to_mrb = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings1 for s2 in seasons for t2 in towers for b2 in buildings2 if (b1,t1,s1)!=(b2,t2,s2) and s1==s2]
intraseason_combinations_to_mrb = list(set(intraseason_combinations_to_mrb)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
intraseason_combinations_to_mrb = random.sample(intraseason_combinations_to_mrb, 3)
print(intraseason_combinations_to_mrb)

for c in intraseason_combinations_to_mrb:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

# Inter-season (Summer to Fall, Fall to Summer)

### Kissam and ESB

In [ ]:
# create all combinations of interseason transfers between summer and fall for Kissam and ESB
buildings = ["ESB", "Kissam"]
towers = [1, 2]
seasons = ["summer", "fall"]
interseason_combinations = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings for s2 in seasons for t2 in towers for b2 in buildings if (b1,t1,s1)!=(b2,t2,s2) and s1!=s2]
interseason_combinations = list(set(interseason_combinations)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
interseason_combinations = random.sample(interseason_combinations, 5)
print(interseason_combinations)

In [ ]:
for c in interseason_combinations:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)

### Testing on MRB

# create all combinations of interseason transfers between summer and fall for transferring from Kissam and ESB to MRB
buildings1 = ["ESB", "Kissam"]
buildings2 = ["MRB"]
towers = [1, 2]
seasons = ["summer", "fall"]
interseason_combinations_to_mrb = [((b1,t1,s1),(b2,t2,s2)) for s1 in seasons for t1 in towers for b1 in buildings1 for s2 in seasons for t2 in towers for b2 in buildings2 if (b1,t1,s1)!=(b2,t2,s2) and s1!=s2]
interseason_combinations_to_mrb = list(set(interseason_combinations_to_mrb)) # making sure there are no duplicates

# TEMPORARY ADDITION FIXME
interseason_combinations_to_mrb = random.sample(interseason_combinations_to_mrb, 3)
print(interseason_combinations_to_mrb)

for c in interseason_combinations_to_mrb:
    b1 = c[0][0]
    t1 = c[0][1]
    s1 = c[0][2]
    b2 = c[1][0]
    t2 = c[1][1]
    s2 = c[1][2]
    print(c)
    plot_multiseed_error_pir(ft_percents=ft_percents, from_building=b1, from_tower=t1, from_season=s1, to_building=b2, to_tower=t2, to_season=s2)